# Assigment Paper:

![image](https://user-images.githubusercontent.com/55464049/114030959-8c7be680-9883-11eb-932b-44146b041417.png)


[Adam](https://arxiv.org/pdf/1412.6980v1.pdf)

[AdamW](https://arxiv.org/abs/1711.05101v3)


[Focal Loss](https://arxiv.org/abs/1708.02002v2)

[Dice](https://arxiv.org/pdf/1707.03237.pdf)

Binary Cross Entropy with logits loss 

# Lec Preperation

##1. Optimizers Read & Summarize 

### 1.1 Adam
a method for efficient stochastic optimization that only requires first-order gradients and requires little memory.

Consists of 2 main methods for reaching fast to the minimum.

1. Momentum: keeping the general direction of the movement. helps avoiding local minimums, and acts like a ball goin down a hill.
![image](https://user-images.githubusercontent.com/55464049/114276061-b244ef00-9a2d-11eb-9c33-feaa59b257a7.png)


2. RMSProp: (a better version of AdaGrad which keeping track of a historical average of square of gradient values)
![image](https://user-images.githubusercontent.com/55464049/114275988-5e3a0a80-9a2d-11eb-854c-d0af3e60f02a.png)
, in RMSProp we also add friction (decay) 
![image](https://user-images.githubusercontent.com/55464049/114275942-14e9bb00-9a2d-11eb-97b8-c200f235ba48.png)
which will not allow the values to explode.
The RMSProp arrests progress along the fast moving direction while simultaneously accelerating progress along slow moving directions, which helps it to bend directly towards the bottom.

![image](https://user-images.githubusercontent.com/55464049/114276907-6d22bc00-9a31-11eb-854a-d903d36e5793.png)



### 1.2 AdamW Optimizer
A simple modification from Adam to recover the original formulation of weight decay regularization by __decoupling__ the weight decay from the optimization steps taken. The weight decay is calculated _after_ the main calculation, and therefore does not interrupt with the them.

[Link](https://towardsdatascience.com/why-adamw-matters-736223f31b5d)
![image](https://user-images.githubusercontent.com/55464049/114277470-fe932d80-9a33-11eb-9158-9967d370654e.png)


### 1.3 SGD
Basically, when we want to progress with our model:
* we can check one example, check the error, compute the gradients and change the network. but, one example can be biased in many ways, and contains an unknown amount of noise. __1 example__.

* So in order to find patterns, and go in the right direction, most likely is that we'll want to check all of our examples, average the direction, compute the loss and backdrop according to all of it.
but, if we have a lot of data (millions-billions..) computing and averaging all the examples for each step will take VERY much time. __N examples__.

* so, we try to find a middle way, here is where __SGD__ comes in! 
We'll randomly subsample our dataset to groups who each have k samples in it.
So rather than computing the loss on the full data-set, in each step we'll compute the loss over a minibatch, and backprop accordingly. __n examples__.

![image](https://user-images.githubusercontent.com/55464049/114276150-2384a200-9a2e-11eb-9c8d-fc12851f536c.png)


## 2. Optimizer Implemetations

### 2.1 AdamW Implementation

In [11]:
# from: https://github.com/egg-west/AdamW-pytorch/blob/master/adamW.py
import math
import torch
from torch.optim.optimizer import Optimizer

class AdamW(Optimizer):
    """Implements Adam algorithm.
    It has been proposed in `Adam: A Method for Stochastic Optimization`_.
    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-3)
        betas (Tuple[float, float], optional): coefficients used for computing
            running averages of gradient and its square (default: (0.9, 0.999))
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        amsgrad (boolean, optional): whether to use the AMSGrad variant of this
            algorithm from the paper `On the Convergence of Adam and Beyond`_
    .. _Adam\: A Method for Stochastic Optimization:
        https://arxiv.org/abs/1412.6980
    .. _On the Convergence of Adam and Beyond:
        https://openreview.net/forum?id=ryQu7f-RZ
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']


                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # if group['weight_decay'] != 0:
                #     grad = grad.add(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1

                # p.data.addcdiv_(-step_size, exp_avg, denom)
                p.data.add_(-step_size,  torch.mul(p.data, group['weight_decay']).addcdiv_(1, exp_avg, denom) )

        return loss


### 2.2 Adam

In [ ]:
import torch
import math
from torch.optim import Adam, SGD

### 2.3 Choose Optimizer:

In [8]:
LR = 0.01
BATCH_SIZE = 32
EPOCH = 12
WD = 0.1

def set_optimizer(opt_name, params, learningRate):
    if opt_name == 'SGD':
        optimizer = optim.SGD(params, lr=learningRate)
    elif opt_name == 'Adam':
        optimizer = optim.Adam(params, lr=learningRate)
    elif opt_name == 'AdamW':
        optimizer = AdamW(params, lr=learningRate, betas=(0.9, 0.99), weight_decay = 0.1)
    else:
        print(f'no {opt_name} optimizer availble, choosing Adam by default.')
        optimizer = optim.Adam(params, lr=learningRate)
    
    if optimizer:
        return optimize

## 3. Losses Read & Summarize 

### 3.1 Cross Entropy - CE
The Cross Entropy function is a way to measure the loss of classification problem. It must be activated after softmax function, which calculates the odds for specific class to be choosen. If the network is 100% sure of a class, it will give it score of 1, and in this case we will have no loss, which means that the NN will not be modified.
The Step size is accoding to the loss, if the loss is high, the CE derivetive will be steep, therefor the step size will be bigger.

[Stat-Quest Video](https://www.youtube.com/watch?v=6ArSys5qHAU&ab_channel=StatQuestwithJoshStarmer)

![image](https://user-images.githubusercontent.com/55464049/114303193-9563f680-9ad5-11eb-9a0b-f6cd93b6ddad.png)


### 3.2 Binary Cross Entropy with logits loss

[What is Logits](https://stackoverflow.com/questions/34240703/what-is-logits-what-is-the-difference-between-softmax-and-softmax-cross-entropy)


* Logits:  the function operates on the unscaled output, that __the values are not probabilities__.

* Binary Cross Entropy: Cross Entropy of only 2 categories.


[Pytorch Link:](https://pytorch.org/docs/master/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.BCEWithLogitsLoss)
This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.



### 3.3 Focal Loss
Focal Loss is a modulated CE function which reduces the loss for easy / common examples, whereas keep the loss for the hard / rare examples.
When __Gamma==0 <=> FocalLoss == CE__ , when gamma 

this approach is useful in many Image Classification Problems.
![image](https://user-images.githubusercontent.com/55464049/114303741-4f5c6200-9ad8-11eb-8edd-5ec9e6cfd095.png)

Another way, apart from Focal Loss, to deal with class imbalance is to introduce weights. Give high weights to the rare class and small weights to the dominating or common class. These weights are referred to as α.

![image](https://user-images.githubusercontent.com/55464049/114303812-aa8e5480-9ad8-11eb-9bf2-b56d8f29620b.png)



### 3.4 Dice

Dice loss is based on the Sorensen-Dice coefficient or Tversky index, which attaches similar importance to false positives and false negatives, and is more immune to the data-imbalance issue. To further alleviate the dominating influence from easy-negative examples in training, we propose to associate training examples with dynamically adjusted weights to deemphasize easy-negative examples.

Dice's formula:
![image](https://user-images.githubusercontent.com/55464049/114304068-e83fad00-9ad9-11eb-9bb9-3ab066dddd02.png)

[Dice impelemtation:](https://www.jeremyjordan.me/semantic-segmentation/#:~:text=around%20the%20cells.%20(-,Source),denotes%20perfect%20and%20complete%20overlap.)

In [15]:
# from : https://www.jeremyjordan.me/semantic-segmentation/#:~:text=around%20the%20cells.%20(-,Source),denotes%20perfect%20and%20complete%20overlap.
def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * np.sum(y_pred * y_true, axes)
    denominator = np.sum(np.square(y_pred) + np.square(y_true), axes)
    
    return 1 - np.mean((numerator + epsilon) / (denominator + epsilon)) # average over classes and batch
    # thanks @mfernezir for catching a bug in an earlier version of this implementation!

## 4. Losses Implementations:



### 4.1  Focal Loss


In [7]:
def focal_loss(pt, alpha, gama):
    return (-1) * alpha * ((1 - pt)**gama) * np.log(pt)

## Snippets

In [13]:
import torch
import math
from torch.optim import Adam, SGD
from torch.optim.optimizer import Optimizer, required

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')


learning_rate = 1e-4

display(model.parameters())
for parm in model.parameters():
    print(f' {parm.shape} \t')
optimizer = AdamW(model.parameters(), lr=learning_rate)
for t in range(500):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

<generator object Module.parameters at 0x7f61f790e2d0>

 torch.Size([100, 1000]) 	
 torch.Size([100]) 	
 torch.Size([10, 100]) 	
 torch.Size([10]) 	


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)


99 62.18852996826172
199 1.151323676109314
299 0.011353669688105583
399 0.000170590152265504
499 1.551429932078463e-06


In [ ]:
%%shell
jupyter nbconvert --to html /content/SecondPracticeML.ipynb